In [1]:
# Oct 13, 2021
# the first dataset becomes too large when these variables are added
# since that at this point we want only to observe the descriptive variables, we can create a secondary dataset

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.delayed import delayed


def data_read(file, names, usecols, dtype):
    df = dd.read_parquet(file, compression='gzip', names=names, usecols=usecols, dtype=dtype, index_col='id', header=0)
    dfs.append(df)

In [3]:
dst='data/dataset2.parquet.gz'

patent='data/cleanpatent.parquet.gz'
# received='data/cit_received.parquet.gz'
# made='data/cit_made.parquet.gz'
# received_delay='data/cit_received_delay.parquet.gz'
# made_delay='data/cit_made_delay.parquet.gz'
# # cit_tree = 'data/cit_tree.csv.gz'
# originality = 'data/originality.parquet.gz'
# generality = 'data/generality.parquet.gz'
# wipo = 'data/wipo.parquet.gz'
# centrality='data/centrality_pagerank.csv'
# int_ext_cit='data/int_ext_cit.parquet.gz'

In [4]:
dfs=[]

#adopting patent.csv as the reference of patents
df = dd.read_parquet(patent, compression='gzip', dtype=object)
df['num_claims']=df['num_claims'].astype(float) #int does not handle NAN values, so using float instead
df.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 8 tasks

In [5]:
#outer join to citation received
#there are patents who received citations that are not in patent.csv.gz
#for example, the very ancient ones

In [6]:
# adding some new files
inventor='data/inventor.parquet.gz'
application='data/application.parquet.gz'
foreign_cit='data/foreigncitation.parquet.gz'
foreign_priority='data/foreign_priority.parquet.gz'
pat_govt='data/df_pat_govt.parquet.gz'
pat_assignee='data/pat_assignee.parquet.gz'
otherref_cit = 'data/otherref_cit.parquet.gz'

In [7]:
df2 = dd.read_parquet(inventor, dtype=object)
df2=df2.set_index('id')
df=df.join(df2, how='outer')
df2.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 7 tasks

In [8]:
df2 = dd.read_parquet(application)
df2=df2.set_index('id')
df=df.join(df2, how='outer')
df2.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 7 tasks

In [9]:
df2 = dd.read_parquet(foreign_cit)
#df2=df2.set_index('patent_id')
df=df.join(df2, how='outer')
df2.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 5 tasks

In [10]:
df2 = dd.read_parquet(foreign_priority, dtype=object)
df2=df2.set_index('patent_id')
df=df.join(df2, how='outer')
df2.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 7 tasks

In [11]:
df2 = dd.read_parquet(pat_govt, dtype=object)
df=df.join(df2, how='outer')
df2.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 5 tasks

In [12]:
df2 = dd.read_parquet(pat_assignee, dtype=object)
#df2=df2.set_index('patent_id')
df=df.join(df2, how='outer')
df2.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 5 tasks

In [13]:
df2 = dd.read_parquet(otherref_cit, dtype=object)
#df2=df2.set_index('patent_id')
df=df.join(df2, how='outer')
df2.memory_usage()

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 5 tasks

In [ ]:
df=df.compute()

In [ ]:
df.info()

In [ ]:
df.sample(n=10)

In [ ]:
df.describe()

In [ ]:
df.describe(include=[np.object]).transpose()

In [ ]:
df.to_parquet(dst, compression='gzip')

In [ ]:
# In[32]:


# this code includes WIPO in the dataset
# as class is also part of non-cited patents, I'm not including in here.
# in any case, for some reason this merge could not be done by the method used in the previous
# there is a bug when using read_csv.gz using dtype and indexing. 
# it is a known bug in the community but i do not know why the previous join did not accused the issue
# the method below corrects the issue 

# names=['index', 'id', 'field_id']
# dtype={'id':object}
# usecols=['id', 'field_id']
# # data_read(wipo, names, usecols, dtype)
# wipo = pd.read_csv.gz(wipo, names=names, usecols=usecols, dtype=dtype, header=0)
# wipo['id']=wipo['id'].astype(str)
# wipo=wipo.set_index('id')
# wipo.info()

# dfs.append(wipo)